In [1]:
import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(gpu, True)
import os

### Input-Pipeline

In [2]:
data = tf.keras.preprocessing.image_dataset_from_directory("PlantVillage/",shuffle = True, seed = 11,
                                                          image_size = (256, 256), 
                                                          batch_size = 32)

Found 2152 files belonging to 3 classes.


In [3]:
class_names = data.class_names
class_names

['potato_early_blight', 'potato_healthy', 'potato_late_blight']

In [4]:
# no of batches in data
len(data)

68

In [5]:
# taking 1 batch
for image, labels in data.take(1):
    print(image.shape)
    print(labels)

(32, 256, 256, 3)
tf.Tensor([2 2 2 1 2 0 2 1 2 0 0 0 2 0 0 0 2 2 2 0 2 2 2 1 0 2 0 0 2 2 1 2], shape=(32,), dtype=int32)


### Train-Test-Split

In [6]:
def dataset_partitions(ds, train_split=0.8, val_split=0.1, test_split=0.1):
    
    ds_size = len(ds)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [7]:
train, val, test = dataset_partitions(data)

In [8]:
train = train.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)
val = val.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)
test = test.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)

### Model-Architecture

In [10]:
resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(256, 256),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
])

In [13]:
INPUT_SHAPE = (32, 256, 256, 3)
NUM_CLASSES = len(class_names)

In [16]:
model = tf.keras.models.Sequential([
    resize_and_rescale,
    tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=INPUT_SHAPE),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'),
])

In [17]:
model.build(INPUT_SHAPE)

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (32, 256, 256, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 254, 254, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (32, 127, 127, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 125, 125, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (32, 62, 62, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (32, 60, 60, 64)          36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (32, 30, 30, 64)         

In [19]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [20]:
history = model.fit(
    train,
    batch_size=32,
    validation_data=val,
    verbose=1,
    epochs=50,
)

Epoch 1/50
54/54 [==============================] - 15s 145ms/step - loss: 0.9620 - accuracy: 0.4413 - val_loss: 0.7626 - val_accuracy: 0.6562
Epoch 2/50
54/54 [==============================] - 5s 90ms/step - loss: 0.6871 - accuracy: 0.6675 - val_loss: 0.6306 - val_accuracy: 0.7396
Epoch 3/50
54/54 [==============================] - 5s 90ms/step - loss: 0.4542 - accuracy: 0.7978 - val_loss: 0.4097 - val_accuracy: 0.8177
Epoch 4/50
54/54 [==============================] - 5s 90ms/step - loss: 0.3587 - accuracy: 0.8543 - val_loss: 0.2116 - val_accuracy: 0.9375
Epoch 5/50
54/54 [==============================] - 5s 91ms/step - loss: 0.3156 - accuracy: 0.8853 - val_loss: 0.1447 - val_accuracy: 0.9479
Epoch 6/50
54/54 [==============================] - 5s 90ms/step - loss: 0.1722 - accuracy: 0.9271 - val_loss: 0.0956 - val_accuracy: 0.9688
Epoch 7/50
54/54 [==============================] - 5s 90ms/step - loss: 0.1210 - accuracy: 0.9519 - val_loss: 0.0598 - val_accuracy: 0.9844
Epoch 8/50


In [21]:
scores = model.evaluate(test)
scores

8/8 [==============================] - 4s 72ms/step - loss: 0.0498 - accuracy: 0.9914


[0.04984382167458534, 0.9913793206214905]

In [37]:
model_version = max([int(i) for i in os.listdir('models/')+[0]])+1
model.save(f"models/{model_version}")
print(f'saved model version {model_version}')

INFO:tensorflow:Assets written to: models/1/assets
saved model version 1
